In [6]:
import pandas as pd
import mysql.connector
import numpy as np

# List of CSV file paths to be loaded into DataFrames
csv_paths = [
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_1kl.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_2ktcl.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_3jksrtc.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_4wbstc.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_5kaac.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_6bsrtc.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_7nbstc.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_8pepsu.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_9ctu.csv",
    r"D:\.intel\.shared\Redbus\Bus Details\df_buses_10wbtc.csv"
]

# Load each CSV file into a DataFrame and store in a list
dfs = [pd.read_csv(path) for path in csv_paths]

# Concatenate all DataFrames into a single DataFrame
Final_df = pd.concat(dfs, ignore_index=True)

# Data cleaning
# Remove "INR" from the Price column, convert to float, and fill missing values with 0
Final_df["Price"] = Final_df["Price"].str.replace("INR", "").astype(float).fillna(0)

# Clean and convert the Rating column to numeric values
Final_df["Rating"] = Final_df["Rating"].str.replace("New", "").str.strip().str.split().str[0]
Final_df["Rating"] = pd.to_numeric(Final_df["Rating"], errors='coerce').fillna(0)

# Filter out rows with Price greater than 7000
Final_df = Final_df[Final_df["Price"] <= 7000]

# Print column names to verify their correctness
print("Columns in Final_df:", Final_df.columns.tolist())

# List of required columns to ensure they are present in the DataFrame
required_columns = ["Bus_name", "Routes_name", "Bus_types", "Start_time", "End_time", "Total_Duration", "Price", "Seats"]

# Remove rows with null values in any of the required columns
Final_df = Final_df.dropna(subset=required_columns)

# Replace NaN values with None
Final_df = Final_df.replace({np.nan: None})

# Debugging: Print the first few rows of the DataFrame to verify data
print(Final_df.head())

# Define the file path for saving the cleaned data to a CSV file
path = r"D:\.intel\.shared\Redbus\Bus Details\Final_busdetails_df.csv"

# Save the cleaned DataFrame to a CSV file
Final_df.to_csv(path, index=False)

# SQL connection and table creation
try:
    # Establish a connection to the MySQL database
    conn = mysql.connector.connect(
        host='localhost',
        user="root",
        password="Pranavu2001",
        port="3306",
        database="RED_BUS_PROJECT"
    )
    
    # Create a cursor object to execute SQL queries
    my_cursor = conn.cursor()

    # Create the database if it does not exist
    my_cursor.execute("CREATE DATABASE IF NOT EXISTS RED_BUS_PROJECT")

    # Create the table 'bus_details' if it does not exist
    my_cursor.execute('''CREATE TABLE IF NOT EXISTS bus_details(
                        ID INT AUTO_INCREMENT PRIMARY KEY,
                        Bus_name VARCHAR(255) NOT NULL,
                        Route_link VARCHAR(255) NULL,
                        Route_name VARCHAR(255) NULL,
                        Bus_type VARCHAR(255) NOT NULL,
                        Price FLOAT NULL,
                        Start_time VARCHAR(255) NOT NULL,
                        End_time VARCHAR(255) NOT NULL,
                        Total_duration VARCHAR(255) NOT NULL,
                        Rating FLOAT NULL,
                        Seats_Available VARCHAR(255) NOT NULL
                        )''')

    # Define the SQL INSERT query
    insert_query = '''INSERT INTO bus_details(
                        Bus_name,
                        Route_link,
                        Route_name,
                        Bus_type,
                        Price,
                        Start_time,
                        End_time,
                        Total_duration,
                        Rating,
                        Seats_Available)
                    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
    
    # Convert the DataFrame to a list of lists for insertion
    data = Final_df.values.tolist()

    # Debugging: Print the first few rows of the data to be inserted
    print(data[:5])

    # Insert data into the table
    my_cursor.executemany(insert_query, data)
    
    # Commit the transaction to save changes
    conn.commit()
    print("Data inserted successfully")

except mysql.connector.Error as err:
    # Print any SQL errors and roll back the transaction if an error occurs
    print(f"Error: {err}")
    conn.rollback()

finally:
    # Close the cursor and connection
    if my_cursor:
        my_cursor.close()
    if conn:
        conn.close()


Columns in Final_df: ['Bus_name', 'Route_link', 'Routes_name', 'Bus_types', 'Price', 'Start_time', 'End_time', 'Total_Duration', 'Rating', 'Seats']
                Bus_name                                         Route_link  \
0    KSRTC (Kerala) - 51  https://www.redbus.in/bus-tickets/bangalore-to...   
1   KSRTC (Kerala) - 789  https://www.redbus.in/bus-tickets/bangalore-to...   
2  KSRTC (Kerala) - 1585  https://www.redbus.in/bus-tickets/bangalore-to...   
3  KSRTC (Kerala) - 1586  https://www.redbus.in/bus-tickets/bangalore-to...   
4  KSRTC (Kerala) - 2133  https://www.redbus.in/bus-tickets/bangalore-to...   

              Routes_name                                 Bus_types  Price  \
0  Bangalore to Kozhikode         Swift Deluxe Non AC Air Bus (2+2)  640.0   
1  Bangalore to Kozhikode            Super Fast Non AC Seater (2+3)  473.0   
2  Bangalore to Kozhikode  Super Deluxe Non AC Seater Air Bus (2+2)  640.0   
3  Bangalore to Kozhikode  Super Deluxe Non AC Seater Air Bus (2+

In [ ]:
# Great! It looks like the data was successfully inserted into the database. Here’s a summary of what was done:

# 1. **Loaded and concatenated the CSV files** into a single DataFrame.
# 2. **Cleaned the data**:
#    - Removed "INR" from the "Price" column and converted it to float.
#    - Extracted and converted the "Rating" to numeric.
#    - Filtered out rows where "Price" exceeded 7000.
# 3. **Handled missing values**:
#    - Dropped rows with null values in required columns.
#    - Replaced remaining NaNs with `None`.
# 4. **Printed the columns** to identify the correct column names.
# 5. **Inserted the cleaned data** into a MySQL table after creating the necessary table structure.

# If you have any further tasks or need additional assistance, feel free to let me know!